In [13]:
import os
import tensorflow as tf
import time

import utils

In [14]:
learning_rate = 0.12
batch_size = 128
n_epochs = 50
n_train = 60000
n_test = 10000

In [15]:
tf.reset_default_graph()

In [16]:
#from tensorflow.examples.tutorials.mnist import input_data
#MNIST = input_data.read_data_sets('data/mnist',one_hot=True)
#print(MNIST.train,MNIST.validation,MNIST.test)

import struct
with open(os.path.join('data/mnist2','train-labels-idx1-ubyte'),'rb') as file:
    print(struct.unpack(">I",file.read(4)))
    print(file.read(8))
    print(type(b'\x00\x00\x08\x01'))
    print(struct.unpack(">II",b'\x00\x00\x08\x01\x00\x00\xea`'))
    print(file.read(8))
    

if you have poor network,then you may need to download the mnist datasets,and extract them manually by using the gzip.open(in_file,'rb') as in,open(out_file,'wb') as out and shutil.copyfileobj(in,out) which is specified in utils.download_one_file

In [17]:
#Step 1: Read in data
#mnist_folder = 'data/mnist2'
mnist_folder = 'data/notmnist'
utils.download_mnist(mnist_folder)
train,val,test = utils.read_mnist(mnist_folder,flatten=True)

data/notmnist\train-images-idx3-ubyte.gz already exists
data/notmnist\train-labels-idx1-ubyte.gz already exists
data/notmnist\t10k-images-idx3-ubyte.gz already exists
data/notmnist\t10k-labels-idx1-ubyte.gz already exists


In [18]:
#Step 2: Create datasets and iterator
#create training Dataset and batch it
train_data= tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000)
train_data = train_data.batch(batch_size)
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

#create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(
    train_data.output_types,
    train_data.output_shapes)
img, label = iterator.get_next()
train_init = iterator.make_initializer(train_data)
test_init = iterator.make_initializer(test_data)


In [19]:
print(img.shape[1],label.shape[1])
w_shape=img.shape[1].value
b_shape = label.shape[1].value

784 10


In [20]:
#step 3:create weights and bias
w1, b1 = tf.get_variable(name='weights1',dtype=tf.float32,initializer = tf.random_normal([w_shape,196])),tf.get_variable(name='bias1',dtype=tf.float32,initializer=tf.random_normal([196]))
w2, b2 = tf.get_variable(name='weights2',dtype=tf.float32,initializer = tf.random_normal([196,64])),tf.get_variable(name='bias2',dtype=tf.float32,initializer=tf.random_normal([64]))
w3, b3 = tf.get_variable(name='weights3',dtype=tf.float32,initializer = tf.random_normal([64,b_shape])),tf.get_variable(name='bias3',dtype=tf.float32,initializer=tf.random_normal([b_shape]))
#step 4:build model
#the model that returns logits,this logits will be later passed through softmax layer
layer1 = tf.matmul(img,w1)+b1
out1 = tf.nn.softmax(layer1)
norm1 = tf.nn.batch_normalization(out1,0,1,None,None,0.001)
layer2 = tf.matmul(norm1,w2)+b2
out2 = tf.nn.softmax(layer2)
norm2 = tf.nn.batch_normalization(out2,0,1,None,None,0.001)
logits = tf.matmul(norm2,w3)+b3
#logits = tf.matmul(layer2,w3)+b3

In [21]:
#step 5:define loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=label,logits=logits)
loss = tf.reduce_mean(entropy)

In [22]:
#step 6:define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01).minimize(loss)

In [23]:
#step 7:calculate accuracy with test set
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds,1),tf.argmax(label,1))
accuracy = tf.reduce_sum(tf.cast(correct_preds,tf.float32))

In [24]:
writer = tf.summary.FileWriter('./graphs/logreg',tf.get_default_graph())
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    #train the model n_epochs times
    for i in range(n_epochs):
        sess.run(train_init)
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer,loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss epoch {0}:{1}'.format(i,total_loss/n_batches))
        print('Total time: {0} seconds'.format(time.time() - start_time))
        
    #test model
    sess.run(test_init)
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    print('Accuracy {0}'.format(total_correct_preds/n_test))
writer.close()
            

Average loss epoch 0:1.4912409032500067
Total time: 6.933532476425171 seconds
Average loss epoch 1:1.0547674894332886
Total time: 9.76781177520752 seconds
Average loss epoch 2:0.9893054723739624
Total time: 12.90245246887207 seconds
Average loss epoch 3:0.9509645955507146
Total time: 15.531826257705688 seconds
Average loss epoch 4:0.8869073603042337
Total time: 18.298013925552368 seconds
Average loss epoch 5:0.8152303981226544
Total time: 21.436826467514038 seconds
Average loss epoch 6:0.7863178285055382
Total time: 24.14909601211548 seconds
Average loss epoch 7:0.7636450717615527
Total time: 26.96174144744873 seconds
Average loss epoch 8:0.7302606262439906
Total time: 29.86602544784546 seconds
Average loss epoch 9:0.6730496232592782
Total time: 32.76762533187866 seconds
Average loss epoch 10:0.6525584255540093
Total time: 35.584447383880615 seconds
Average loss epoch 11:0.6449329026216685
Total time: 38.160019874572754 seconds
Average loss epoch 12:0.6330169646545898
Total time: 40.68